In [2]:
import cv2
import numpy as np
import mediapipe as mp
import tensorflow as tf
from tensorflow.keras.models import load_model
from random import randint

In [3]:
# initialize mediapipe
mpHands = mp.solutions.hands
hands = mpHands.Hands(max_num_hands=1, min_detection_confidence=0.7)
mpDraw = mp.solutions.drawing_utils

In [4]:
# Load the gesture recognizer model
model = load_model('../models/hand-gesture-recognition-code/mp_hand_gesture')

# Load class names
f = open('../models/hand-gesture-recognition-code/gesture.names', 'r')
classNames = f.read().split('\n')
f.close()




['okay', 'scissors', 'thumbs up', 'thumbs down', 'call me', 'paper', 'rock', 'live long', 'rock', 'smile']


In [5]:
def initializeVideoCapture():
    cap = cv2.VideoCapture(0)
    return cap

In [6]:
def rockPaperScissors():
    gestures = {
        0: "rock",
        1: "paper",
        2: "scissors"
    }

    result = randint(0, 1000)  % 3
    return gestures[result]

In [7]:
def gameResult(user, computer):
    if user == computer:
        return -1
    elif user == "rock":
        if computer == "paper":
            return 0
        else:
            return 1
    elif user == "paper":
        if computer == "scissors":
            return 0
        else:
            return 1
    elif user == "scissors":
        if computer == "Rock":
            return 0
        else:
            return 1

In [8]:
def countdown(frame, seconds):
    x, y, _ = frame.shape
    for i in range(seconds, 0, -1):
        cv2.rectangle(frame, (0, 0), (y, x), (0, 0, 0), -1)
        centerText(str(i), frame)
        cv2.imshow("Rock Paper Scissors", frame)
        cv2.waitKey(1000)


In [9]:
def centerText(msg, frame, colour=(255, 255, 255)):
    text = msg
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 1
    font_thickness = 2

    # Get the size of the text
    text_size = cv2.getTextSize(text, font, font_scale, font_thickness)[0]

    # Calculate the position to place the text in the middle
    text_x = (frame.shape[1] - text_size[0]) // 2
    text_y = (frame.shape[0] + text_size[1]) // 2

    # Put the text on the image
    cv2.putText(frame, text, (text_x, text_y), font, font_scale, colour, font_thickness)

    return frame

In [14]:
def processGesture(frame, hands, model, classNames):
    x, y, _ = frame.shape
    frame = cv2.flip(frame, 1)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    result = hands.process(frame_rgb)
    msg = "No hand detected. Try again!"
    className = None

    if result.multi_hand_landmarks:
        landmarks = []
        for hand_landmarks in result.multi_hand_landmarks:
            for lm in hand_landmarks.landmark:
                lmx = int(lm.x * x)
                lmy = int(lm.y * y)
                landmarks.append([lmx, lmy])

        prediction = model.predict([landmarks])
        classID = np.argmax(prediction)
        className = classNames[classID]

        if className not in ["rock", "paper", "scissors"]:
            msg = "Invalid gesture. Try again!"
        else:
            msg = processGameResult(className)

    return className, msg

In [11]:
def processGameResult(player_gesture):
    opponent = rockPaperScissors()
    result = gameResult(player_gesture, opponent)
    if result == -1:
        msg_result = "It's a tie!"
    elif result == 0:
        msg_result = "You lost!"
    else:
        msg_result = "You won!"
    
    return f"I showed {opponent}. {msg_result}!"

In [12]:
userScore = 0
computerScore = 0

# Initialize the webcam
cap = initializeVideoCapture()

while True:
    # Read each frame from the webcam
    _, frame = cap.read()

    x, y, c = frame.shape

    # Flip the frame vertically
    frame = cv2.flip(frame, 1)
    framergb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # Show the score in the bottom left
    cv2.putText(frame, f"Score: {userScore} - {computerScore}", (10, x - 25), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    cv2.imshow("Rock Paper Scissors", frame) 

    # If spacebar is pressed
    if cv2.waitKey(1) == ord(' '):
        msg = "Get ready!"

        # Show black screen with message in the middle of the screen
        cv2.rectangle(frame, (0, 0), (y, x), (0, 0, 0), -1)
        frame = centerText(msg, frame)
        cv2.imshow("Rock Paper Scissors", frame)
        cv2.waitKey(2000)

        countdown(frame, 3)
        
        className = None

        _, frame = cap.read()

        # Process the gesture
        player_gesture, result_message = processGesture(frame, hands, model, classNames)

        # show the result
        cv2.putText(frame, f"You showed {className}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        frame = centerText(result_message, frame, colour=(255, 0, 0))
        
        # Show the final output
        cv2.imshow("Rock Paper Scissors", frame) 
        cv2.waitKey(5000)
            
    elif cv2.waitKey(1) == ord('q'): 
        break
    else:
        continue
    
        
# release the webcam and destroy all active windows
cap.release()

cv2.destroyAllWindows()

1/1 [==============================] - 0s 434ms/step
